<a href="https://colab.research.google.com/github/LuisDi98/Audio_a_Lesco/blob/dev/Audio_To_LESCO_models_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Install dependencies
! pip install datasets
! pip install transformers
! pip install -U openai-whisper
! pip install git+https://github.com/openai/whisper.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 KB 21.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 KB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796926 sha256=f9af4aead7d25a36c39731d555c97204f63021d17e4cc4d395c510e262e25279
  Stored in directory: /root/.cache

# Get Dataset
CIEMPIESS_TEST

In [4]:
#Load the dataset
from datasets import load_dataset, load_metric, Audio
import whisper
ds=load_dataset("ciempiess/ciempiess_test", split="test")

#Downsample to 16kHz
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset ciempiess_test downloaded and prepared to /root/.cache/huggingface/datasets/ciempiess___ciempiess_test/ciempiess_test/1.0.0/6308ec4b2e9db78ec4aa6169117a8b2e2b3ac399c99501e57de918b528539b0b. Subsequent calls will reuse this data.


In [6]:
ds

Dataset({
    features: ['audio_id', 'audio', 'speaker_id', 'gender', 'duration', 'normalized_text'],
    num_rows: 3558
})

In [11]:
ds[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/024bce9383676d0a3a6afc0dd8423b344044040363dd48ff5402220707a9d22c/test/male/M_07/CMPT_M_07_0074.flac',
 'array': array([-0.00192261, -0.00234985, -0.00158691, ..., -0.00839233,
        -0.00900269, -0.00698853]),
 'sampling_rate': 16000}

#Whisper-large
* De momento no ocupamos los inputs, esta es una manera más sencilla.
* Queda pendiente probar wav2vec2-large-xlsr-53-spanish-ep5-944h: https://huggingface.co/carlosdanielhernandezmena/wav2vec2-large-xlsr-53-spanish-ep5-944h
* Comparar con el word error rate


In [9]:
model = whisper.load_model("large")
# Transcribe from audio to text with the first sample
sample = ds[0]["audio"]["path"]
result = model.transcribe(sample)
print("Whisper result: ",result["text"])
print("Normalized_text of the sample: ",ds[0]["normalized_text"])

100%|█████████████████████████████████████| 2.87G/2.87G [00:57<00:00, 54.1MiB/s]
/usr/local/lib/python3.9/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Whisper result:   Pues ahí está la libertad de las posiciones, de a ver quién es pasivo, quién es activo, bla, bla, bla. Muchas cosas, ¿no? Pero...
Normalized_text of the sample:  pues está la libertá de las posiciones de a ver quién es pasivo quién es activo blablablá muchas cosas no pero


#Whisper-medium

###TO DO: 
Usar el dataset en espaniol y probar el modelo.

Buscar una metrica adecuada y una biblioteca para el testing

Ver porque esta dando samples en ingles y no espaniol al usar el dataset de ciempiess

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-medium")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="spanish", task="transcribe")


# load dummy dataset and read audio files
ds=load_dataset("ciempiess/ciempiess_test", split="test")
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))
sample = ds[0]["audio"]
input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features 
print(input_features)
# generate token ids
predicted_ids = model.generate(input_features)
print(predicted_ids)
# decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)
print(transcription)

transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
print(transcription)


tensor([[[-0.1399,  0.5492,  0.2900,  ..., -0.7534, -0.7534, -0.7534],
         [ 0.2928,  0.3964,  0.2335,  ..., -0.7534, -0.7534, -0.7534],
         [ 0.2252,  0.2252,  0.0790,  ..., -0.7534, -0.7534, -0.7534],
         ...,
         [-0.6674, -0.6745, -0.5624,  ..., -0.7534, -0.7534, -0.7534],
         [-0.5837, -0.6500, -0.5296,  ..., -0.7534, -0.7534, -0.7534],
         [-0.5518, -0.7030, -0.6630,  ..., -0.7534, -0.7534, -0.7534]]])


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[50258, 50259, 50359, 50363,  1042,    11,   456,   307,   264,  5645,
           295,   264,  8432,    11,   295,  2577,   567,   307, 14975,    11,
           567,   307,  4967,    11, 12288, 12288, 12288,    11,   867,   721,
            11,   558,    30,   583,   485, 50257]])
['<|startoftranscript|><|en|><|transcribe|><|notimestamps|> Well, there is the freedom of the positions, of seeing who is passive, who is active, blah blah blah, many things, right? But...<|endoftext|>']
[' Well, there is the freedom of the positions, of seeing who is passive, who is active, blah blah blah, many things, right? But...']


#Whisper-tiny